# Selecting and Tuning pipelines

This guide shows you how to search for multiple pipelines for your problem
and later on use a [BTBSession](https://hdi-project.github.io/BTB/api/btb.session.html#btb.session.BTBSession)
to select and tune the best one.

Note that some steps are not explained for simplicity. Full details
about them can be found in the previous parts of the tutorial.

Here we will:

1. Load a dataset
2. Search and load suitable templates
3. Write a scoring function
4. Build a BTBSession for our templates
5. Run the session to find the best pipeline

## Load the Dataset

The first step will be to load the dataset.

In [1]:
from mlprimitives.datasets import load_dataset

dataset = load_dataset('census')

In [2]:
dataset.describe()

Adult Census dataset.

    Predict whether income exceeds $50K/yr based on census data. Also known as "Adult" dataset.

    Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean
    records was extracted using the following conditions: ((AAGE>16) && (AGI>100) &&
    (AFNLWGT>1)&& (HRSWK>0))

    Prediction task is to determine whether a person makes over 50K a year.

    source: "UCI
    sourceURI: "https://archive.ics.uci.edu/ml/datasets/census+income"
    
Data Modality: single_table
Task Type: classification
Task Subtype: binary
Data shape: (32561, 14)
Target shape: (32561,)
Metric: accuracy_score
Extras: 


## Find and load suitable Templates

We will be using the `mlblocks.discovery.find_pipelines` function to search
for compatible pipelines.

In this case, we will be looking for `single_table/classification` pipelines.

In [3]:
from mlblocks.discovery import find_pipelines

filters = {
    'metadata.data_modality': 'single_table',
    'metadata.task_type': 'classification'
}
templates = find_pipelines(filters=filters)

In [4]:
templates

['single_table.classification.categorical_encoder.logit',
 'single_table.classification.categorical_encoder.random_forest',
 'single_table.classification.categorical_encoder.xgboost',
 'single_table.classification.mlprimitives.logit',
 'single_table.classification.mlprimitives.random_forest',
 'single_table.classification.mlprimitives.xgboost',
 'single_table.classification.mlprimitives_text.xgboost']

And we will create a dictionary with MLPipeline instances that will be used as tempaltes for our tuning.

In [5]:
from mlblocks import MLPipeline

templates_dict = {
    template: MLPipeline(template)
    for template in templates
}

In [6]:
templates_dict['single_table.classification.mlprimitives.xgboost']

## Create a scoring function

In order to use a `BTBSession` we will need a function that is able to score a proposal,
which will always be a pair of template name and proposed hyperparameters.

In this case, the evaluation will be done using 5-fold cross validation over our dataset.

In [7]:
import numpy as np

def cross_validate(template_name, hyperparameters=None):
    template = templates_dict[template_name]
    scores = []
    for X_train, X_test, y_train, y_test in dataset.get_splits(5):
        pipeline = MLPipeline(template.to_dict())  # Make a copy of the template
        if hyperparameters:
            pipeline.set_hyperparameters(hyperparameters)

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        
        scores.append(dataset.score(y_test, y_pred))
        
    return np.mean(scores)

## Setup the BTBSession

We will create another dictionary with the tunable hyperparameters of each template.
This will be used by the BTBSession to know how to tune each template.

In [8]:
tunables = {
    name: template.get_tunable_hyperparameters(flat=True)
    for name, template in templates_dict.items()
}

In [9]:
tunables['single_table.classification.mlprimitives.xgboost']

{('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
  'max_labels'): {'type': 'int', 'default': 0, 'range': [0, 100]},
 ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
  'lowercase'): {'type': 'bool', 'default': True},
 ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
  'binary'): {'type': 'bool', 'default': True},
 ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
  'max_features'): {'type': 'int', 'default': 1000, 'range': [1, 10000]},
 ('sklearn.impute.SimpleImputer#1', 'strategy'): {'type': 'str',
  'default': 'mean',
  'values': ['mean', 'median', 'most_frequent', 'constant']},
 ('xgboost.XGBClassifier#1', 'n_estimators'): {'type': 'int',
  'default': 100,
  'range': [10, 1000]},
 ('xgboost.XGBClassifier#1', 'max_depth'): {'type': 'int',
  'default': 3,
  'range': [3, 10]},
 ('xgboost.XGBClassifier#1', 'learning_rate'): {'type': 'float',
  'default': 0.1,
  'range': [0, 1]},
 ('xgboost.XGBClassifier#1', 'gamma'): {'type': 'float'

And then create a `BTBSession` instance passing them and the `cross_validate` function.

We will also be setting it in `verbose` mode, so we can have a better insight on what is going on.

In [11]:
from btb.session import BTBSession

session = BTBSession(tunables, cross_validate, verbose=True)

## 5. Run the session

After everything is set up, we can start running the tuning session passing it
the number of iterations that we want to perform.

In [12]:
session.run(5)

2020-01-23 20:16:01,059 - INFO - session - Creating Tunable instance from dict.
2020-01-23 20:16:01,060 - INFO - session - Obtaining default configuration for single_table.classification.categorical_encoder.logit
2020-01-23 20:16:03,274 - INFO - session - New optimal found: single_table.classification.categorical_encoder.logit - 0.7975185708718643
2020-01-23 20:16:03,284 - INFO - session - Creating Tunable instance from dict.
2020-01-23 20:16:03,285 - INFO - session - Obtaining default configuration for single_table.classification.categorical_encoder.random_forest
2020-01-23 20:16:05,584 - INFO - session - Creating Tunable instance from dict.
2020-01-23 20:16:05,585 - INFO - session - Obtaining default configuration for single_table.classification.categorical_encoder.xgboost
2020-01-23 20:16:10,613 - INFO - session - New optimal found: single_table.classification.categorical_encoder.xgboost - 0.8639171383183359
2020-01-23 20:16:10,617 - INFO - session - Creating Tunable instance from d

{'id': '51a54054874dd7a83ff0e785ffdfee3b',
 'name': 'single_table.classification.categorical_encoder.xgboost',
 'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
   'max_labels'): 0,
  ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
  ('xgboost.XGBClassifier#1', 'n_estimators'): 100,
  ('xgboost.XGBClassifier#1', 'max_depth'): 3,
  ('xgboost.XGBClassifier#1', 'learning_rate'): 0.1,
  ('xgboost.XGBClassifier#1', 'gamma'): 0.0,
  ('xgboost.XGBClassifier#1', 'min_child_weight'): 1},
 'score': 0.8639171383183359}

During this loop, the BTBSession will build pipelines based on our templates and evaluate them
using our scoring function.

## 6. Evaluate results

When the session funishes running it will return a the best proposal available and the
obtained score.

These results are also available as the `best_proposal` attribute from the btb session object.

In [14]:
session.best_proposal

{'id': '51a54054874dd7a83ff0e785ffdfee3b',
 'name': 'single_table.classification.categorical_encoder.xgboost',
 'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
   'max_labels'): 0,
  ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
  ('xgboost.XGBClassifier#1', 'n_estimators'): 100,
  ('xgboost.XGBClassifier#1', 'max_depth'): 3,
  ('xgboost.XGBClassifier#1', 'learning_rate'): 0.1,
  ('xgboost.XGBClassifier#1', 'gamma'): 0.0,
  ('xgboost.XGBClassifier#1', 'min_child_weight'): 1},
 'score': 0.8639171383183359}

## Continue Running

If we feel that the score can still be improved and want to keep searching, we can simply run the session again which will continue tuning over the previous results.

In [15]:
session.run(20)

2020-01-23 20:17:59,163 - INFO - session - Creating Tunable instance from dict.
2020-01-23 20:17:59,163 - INFO - session - Obtaining default configuration for single_table.classification.mlprimitives.xgboost
2020-01-23 20:18:04,640 - INFO - session - Creating Tunable instance from dict.
2020-01-23 20:18:04,640 - INFO - session - Obtaining default configuration for single_table.classification.mlprimitives_text.xgboost
2020-01-23 20:18:04,779 - ERROR - mlpipeline - Exception caught producing MLBlock mlprimitives.custom.text.TextCleaner#1
Traceback (most recent call last):
  File "/home/xals/.virtualenvs/MLBlocks/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2657, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1601, in pandas._libs.hashtable.PyObject

2020-01-23 20:19:13,340 - WARNING - session - Too many errors: 1. Removing tunable single_table.classification.categorical_encoder.logit
2020-01-23 20:19:13,343 - INFO - session - Generating new proposal configuration for single_table.classification.mlprimitives.logit
2020-01-23 20:19:26,076 - INFO - session - Generating new proposal configuration for single_table.classification.categorical_encoder.random_forest
2020-01-23 20:19:31,573 - INFO - session - Generating new proposal configuration for single_table.classification.mlprimitives.random_forest
2020-01-23 20:19:34,763 - INFO - session - Generating new proposal configuration for single_table.classification.categorical_encoder.xgboost
2020-01-23 20:20:15,775 - INFO - session - Generating new proposal configuration for single_table.classification.mlprimitives.xgboost
2020-01-23 20:21:49,655 - INFO - session - Generating new proposal configuration for single_table.classification.mlprimitives.logit
2020-01-23 20:21:49,946 - ERROR - mlp

{'id': 'd9854a57d48100da0f3584dc4490301f',
 'name': 'single_table.classification.mlprimitives.xgboost',
 'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
   'max_labels'): 22,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'lowercase'): True,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'binary'): True,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'max_features'): 3863,
  ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
  ('xgboost.XGBClassifier#1', 'n_estimators'): 193,
  ('xgboost.XGBClassifier#1', 'max_depth'): 3,
  ('xgboost.XGBClassifier#1', 'learning_rate'): 0.29839198565184866,
  ('xgboost.XGBClassifier#1', 'gamma'): 0.19826736959824165,
  ('xgboost.XGBClassifier#1', 'min_child_weight'): 4},
 'score': 0.8739290413691612}

**NOTE**: If you look at the logs you will notice how the BTBSession captures the errors that finds
while executing the pipelines and automatically discards the failing tempaltes to be able to continue
the tuning session without wasting time on them.

The number of errors that we want to wait before discarding a template can be changed passing the
`max_errors` argument to the `BTBSession` when it is build.

Isn't it cool?

## Build the best pipeline

Once we are satisfied with the results, we can then build an instance of the best pipeline
by reading the `best_proposal` attribute from the `session`.

In [16]:
best_proposal = session.best_proposal
best_proposal

{'id': 'd9854a57d48100da0f3584dc4490301f',
 'name': 'single_table.classification.mlprimitives.xgboost',
 'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
   'max_labels'): 22,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'lowercase'): True,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'binary'): True,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'max_features'): 3863,
  ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
  ('xgboost.XGBClassifier#1', 'n_estimators'): 193,
  ('xgboost.XGBClassifier#1', 'max_depth'): 3,
  ('xgboost.XGBClassifier#1', 'learning_rate'): 0.29839198565184866,
  ('xgboost.XGBClassifier#1', 'gamma'): 0.19826736959824165,
  ('xgboost.XGBClassifier#1', 'min_child_weight'): 4},
 'score': 0.8739290413691612}

In [17]:
template = templates_dict[best_proposal['name']]

pipeline = MLPipeline(template.to_dict())
pipeline.set_hyperparameters(best_proposal['config'])

pipeline.fit(dataset.data, dataset.target)

## Explore other results

Optionally, if we are interested in exploring the results of the previous proposals we can access them
in the `trials` attribute of the `session` object.

In [20]:
list(session.proposals.values())[0:2]

[{'id': '9dd9a11254f46b11ad42a12692b4965e',
  'name': 'single_table.classification.categorical_encoder.logit',
  'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
    'max_labels'): 0,
   ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
   ('sklearn.linear_model.LogisticRegression#1', 'fit_intercept'): True,
   ('sklearn.linear_model.LogisticRegression#1', 'max_iter'): 100,
   ('sklearn.linear_model.LogisticRegression#1', 'solver'): 'liblinear',
   ('sklearn.linear_model.LogisticRegression#1', 'penalty'): 'l2',
   ('sklearn.linear_model.LogisticRegression#1', 'C'): 1.0,
   ('sklearn.linear_model.LogisticRegression#1', 'multi_class'): 'ovr',
   ('sklearn.linear_model.LogisticRegression#1', 'intercept_scaling'): 1.0,
   ('sklearn.linear_model.LogisticRegression#1', 'tol'): 0.0001,
   ('sklearn.linear_model.LogisticRegression#1', 'dual'): False},
  'score': 0.7975185708718643},
 {'id': 'f7ef0814341cee4f05280077b9b3de9c',
  'name': 'single_table.classificati